In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import json # Importa a biblioteca json para trabalhar com dados JSON

def obter_generos_oscar(anos):
    todos_os_generos = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for ano in anos:
        url = f"https://www.imdb.com/pt/event/ev0000003/{ano}/1/?ref_=ev_acd_tl_yr_1"
        print(f"Coletando dados para o Oscar de {ano} - URL: {url}")

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            generos_ano = {'Ano': ano,
                           'Gênero do Melhor Filme': 'Não encontrado',
                           'Gênero do Melhor Diretor': 'Não encontrado',
                           'Gênero do Melhor Ator': 'Não encontrado',
                           'Gênero do Melhor Atriz': 'Não encontrado',
                           'Gênero do Melhor Ator Coadjuvante': 'Não encontrado',
                           'Gênero do Melhor Atriz Coadjuvante': 'Não encontrado'}

            categorias = {
                'Melhor Filme': 'BestMotionPictureoftheYear',
                'Melhor Diretor': 'BestAchievementinDirecting',
                'Melhor Ator': 'BestPerformancebyanActorinaLeadingRole',
                'Melhor Atriz': 'BestPerformancebyanActressinaLeadingRole',
                'Melhor Ator Coadjuvante': 'BestPerformancebyanActorinaSupportingRole',
                'Melhor Atriz Coadjuvante': 'BestPerformancebyanActressinaSupportingRole'
            }

            for categoria_nome, data_testid in categorias.items():
                category_container = soup.find('section', {'data-testid': data_testid})

                if category_container:
                    movie_link_tag = None
                    if categoria_nome == 'Melhor Filme':
                        winner_li = category_container.find('li')
                        if winner_li:
                            movie_link_tag = winner_li.find('a', href=True)
                    else:
                        winner_li = category_container.find('li')
                        if winner_li:
                            all_ul_in_winner = winner_li.find_all('ul', class_='ipc-inline-list')
                            if len(all_ul_in_winner) > 1:
                                movie_link_tag = all_ul_in_winner[-1].find('a', href=True)
                            else:
                                movie_link_tag = winner_li.find('a', href=True)

                    if movie_link_tag:
                        movie_url = "https://www.imdb.com" + movie_link_tag['href']
                        print(f"  Link de filme para '{categoria_nome}' encontrado: {movie_url}")
                        time.sleep(1)
                        try:
                            movie_response = requests.get(movie_url, headers=headers)
                            movie_response.raise_for_status()
                            movie_soup = BeautifulSoup(movie_response.text, 'html.parser')

                            genres = []
                            
                            # NOVA TENTATIVA: Buscar gêneros do JSON-LD
                            script_tag = movie_soup.find('script', type='application/ld+json')
                            if script_tag:
                                try:
                                    json_data = json.loads(script_tag.string)
                                    if 'genre' in json_data:
                                        # O campo 'genre' pode ser uma string ou uma lista de strings
                                        if isinstance(json_data['genre'], list):
                                            genres = json_data['genre']
                                        elif isinstance(json_data['genre'], str):
                                            genres = [json_data['genre']] # Converte para lista se for string única
                                except json.JSONDecodeError as e:
                                    print(f"    Erro ao decodificar JSON-LD da página do filme ({movie_url}): {e}")
                                except Exception as e:
                                    print(f"    Erro inesperado ao processar JSON-LD da página do filme ({movie_url}): {e}")

                            if genres:
                                generos_ano[f'Gênero do {categoria_nome}'] = ", ".join(genres)
                            else:
                                print(f"    Gêneros para '{categoria_nome}' não encontrados na URL do filme: {movie_url}")
                                # Comente esta linha novamente se não quiser ver o HTML bruto:
                                # print(f"    HTML da página do filme ({movie_url}):\n{movie_response.text[:5000]}...\n") 
                                

                        except requests.exceptions.RequestException as e:
                            print(f"    Erro ao acessar URL do filme para gêneros ({movie_url}): {e}")
                        except Exception as e:
                            print(f"    Erro inesperado ao processar gêneros do filme ({movie_url}): {e}")
                    else:
                        print(f"  Link de filme para '{categoria_nome}' NÃO encontrado para o Oscar de {ano}.")
                else:
                    print(f"  Contêiner da categoria para '{categoria_nome}' NÃO encontrado para o Oscar de {ano}.")

            todos_os_generos.append(generos_ano)
            time.sleep(2)
        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar URL do Oscar {ano}: {e}")
        except Exception as e:
            print(f"Erro inesperado ao processar o Oscar de {ano}: {e}")

    df_generos = pd.DataFrame(todos_os_generos)
    return df_generos

# Exemplo de uso
anos_para_coletar = range(2005, 2026)
df_generos_vencedores = obter_generos_oscar(anos_para_coletar)

# Salvar os dados
df_generos_vencedores.to_csv('oscar_generos_vencedores_2005_em_diante.csv', index=False)
print("\nDados de gênero salvos em 'oscar_generos_vencedores_2005_em_diante.csv'")
print(f"Total de anos com dados de gênero coletados: {len(df_generos_vencedores)}")

print("\n--- Primeiras 25 linhas dos dados de Gênero coletados ---")
print(df_generos_vencedores.head(25).to_markdown(index=False))

Coletando dados para o Oscar de 2005 - URL: https://www.imdb.com/pt/event/ev0000003/2005/1/?ref_=ev_acd_tl_yr_1
  Link de filme para 'Melhor Filme' encontrado: https://www.imdb.com/pt/title/tt0405159/?ref_=ev_i_1
  Link de filme para 'Melhor Diretor' encontrado: https://www.imdb.com/pt/title/tt0405159/?ref_=ev_nom
  Link de filme para 'Melhor Ator' encontrado: https://www.imdb.com/pt/title/tt0350258/?ref_=ev_nom
  Link de filme para 'Melhor Atriz' encontrado: https://www.imdb.com/pt/title/tt0405159/?ref_=ev_nom
  Link de filme para 'Melhor Ator Coadjuvante' encontrado: https://www.imdb.com/pt/title/tt0405159/?ref_=ev_nom
  Link de filme para 'Melhor Atriz Coadjuvante' encontrado: https://www.imdb.com/pt/title/tt0338751/?ref_=ev_nom
Coletando dados para o Oscar de 2006 - URL: https://www.imdb.com/pt/event/ev0000003/2006/1/?ref_=ev_acd_tl_yr_1
  Link de filme para 'Melhor Filme' encontrado: https://www.imdb.com/pt/title/tt0375679/?ref_=ev_i_1
  Link de filme para 'Melhor Diretor' encontr